# Лабораторная работа №10

In [3]:
!pip install pandas plotly streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 71.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.7 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st

# Для Streamlit через ngrok
from pyngrok import ngrok

In [43]:
import requests
import pandas as pd

url='https://api.hh.ru/vacancies'
cities=['https://api.hh.ru/areas/1',
        'https://api.hh.ru/areas/2',
        'https://api.hh.ru/areas/3',
        'https://api.hh.ru/areas/4']
df=pd.DataFrame()
for j in range(5):
  for i in range(20):
    params={'text':'Аналитик',
          'search_field':'name',
          'per_page':100,
          'only_with_salary':True,
          'page':i}
    try:
      js=requests.get(url,params=params).json()['items']
      print(requests.get(url,params=params).json()['found'])
      tmp=pd.json_normalize(js)[['id','name','area.id','area.url','area.name','salary.from',
                                'salary.to','salary.currency','salary.gross',
                                'experience.name']]
      tmp=tmp[tmp['area.url'].isin(cities)]
      df=pd.concat([df,tmp])
      df.drop_duplicates(inplace=True)
    except:
      break
df.info()

4069
4069
4069
4070
4069
4069
4070
4069
4070
4070
4069
4070
4069
4070
4069
4070
4070
4070
4070
4069
4070
4069
4069
4069
4069
4070
4070
4069
4069
4069
4070
4070
4069
4069
4070
4070
4069
4069
4069
4069
4069
4069
4069
4069
4070
4069
4069
4070
4069
4070
4069
4070
4069
4069
4069
4069
4069
4069
4070
4069
4069
4069
4069
4069
4069
4069
4069
4070
4069
4070
4070
4069
4069
4069
4070
4069
4069
4069
4069
4069
4069
<class 'pandas.core.frame.DataFrame'>
Index: 1075 entries, 2 to 98
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               1075 non-null   object 
 1   name             1075 non-null   object 
 2   area.id          1075 non-null   object 
 3   area.url         1075 non-null   object 
 4   area.name        1075 non-null   object 
 5   salary.from      897 non-null    float64
 6   salary.to        614 non-null    float64
 7   salary.currency  1075 non-null   object 
 8   salary.gross     1075 non-null

In [46]:
# Подсчет Зарплаты
def calculate_salary(row):
    salary_from = row.get('salary.from', np.nan)
    salary_to = row.get('salary.to', np.nan)
    salary_gross = row.get('salary.gross', False)

    if pd.isnull(salary_from) and pd.isnull(salary_to):
        return np.nan
    elif pd.isnull(salary_from):
        average_salary = salary_to
    elif pd.isnull(salary_to):
        average_salary = salary_from
    else:
        average_salary = (salary_from + salary_to) / 2

    coefficient = 1 if not salary_gross else 0.87
    return average_salary * coefficient

df['Зарплата'] = df.apply(calculate_salary, axis=1)

df.head()

,id,name,area.id,area.url,area.name,salary.from,salary.to,salary.currency,salary.gross,experience.name,salary,Зарплата
2,112312336,Инвестиционный аналитик,2,https://api.hh.ru/areas/2,Санкт-Петербург,150000.0,150000.0,RUR,False,От 1 года до 3 лет,150000.0,150000.0
3,113281092,Ассистент аналитика,1,https://api.hh.ru/areas/1,Москва,80000.0,NaN,RUR,False,Нет опыта,80000.0,80000.0
4,113740571,System Analyst/Системный Аналитик (Стажер/Junior),2,https://api.hh.ru/areas/2,Санкт-Петербург,NaN,100000.0,RUR,False,Нет опыта,100000.0,100000.0
5,109225941,Системный аналитик,1,https://api.hh.ru/areas/1,Москва,150000.0,180000.0,RUR,False,От 1 года до 3 лет,165000.0,165000.0
6,111899122,Аналитик данных,1,https://api.hh.ru/areas/1,Москва,250000.0,NaN,RUR,True,Нет опыта,217500.0,217500.0


In [22]:
# Подсчет квалификации

def classify_qualification(row):
    experience = row.get('experience.name', '').strip()
    name = row.get('name', '').lower()

    if (experience in ["Более 6 лет", "От 3 до 6 лет"]) and any(keyword in name for keyword in ["senior", "ведущий", "старший"]):
        return "Сеньор"
    elif experience == "От 1 года до 3 лет":
        return "Миддл"
    elif (experience == "Нет опыта") and any(keyword in name for keyword in ["начинающий", "стажер", "помощник"]):
        return "Джун"
    else:
        return "Другое"

df['Квалификация'] = df.apply(classify_qualification, axis=1)

df.head()


,id,name,area.id,area.url,area.name,salary.from,salary.to,salary.currency,salary.gross,experience.name,Зарплата,Квалификация
2,112312336,Инвестиционный аналитик,2,https://api.hh.ru/areas/2,Санкт-Петербург,150000.0,150000.0,RUR,False,От 1 года до 3 лет,150000.0,Миддл
3,113281092,Ассистент аналитика,1,https://api.hh.ru/areas/1,Москва,80000.0,NaN,RUR,False,Нет опыта,80000.0,Другое
4,113740571,System Analyst/Системный Аналитик (Стажер/Junior),2,https://api.hh.ru/areas/2,Санкт-Петербург,NaN,100000.0,RUR,False,Нет опыта,100000.0,Джун
5,109225941,Системный аналитик,1,https://api.hh.ru/areas/1,Москва,150000.0,180000.0,RUR,False,От 1 года до 3 лет,165000.0,Миддл
6,111899122,Аналитик данных,1,https://api.hh.ru/areas/1,Москва,250000.0,NaN,RUR,True,Нет опыта,217500.0,Другое


In [23]:
# Круговая диаграмма доли вакансий в разных городах
city_counts = df['area.name'].value_counts().reset_index()
city_counts.columns = ['Город', 'Количество вакансий']

fig = px.pie(city_counts, names='Город', values='Количество вакансий',
             title='Доля вакансий по городам', color_discrete_sequence=px.colors.sequential.RdBu)
fig.show()

In [25]:
# Диаграмма рассеяния зависимости зарплаты от числа лет опыта
def extract_years(experience):
    if experience == "Нет опыта":
        return 0
    elif experience == "От 1 года до 3 лет":
        return 2
    elif experience == "От 3 до 6 лет":
        return 4.5
    elif experience == "Более 6 лет":
        return 7
    else:
        return np.nan

df['Опыт_лет'] = df['experience.name'].apply(extract_years)

scatter_df = df.dropna(subset=['Зарплата', 'Опыт_лет'])

fig = px.scatter(scatter_df, x='Опыт_лет', y='Зарплата',
                 hover_data=['area.name', 'Квалификация'],
                 title='Зависимость зарплаты от опыта работы',
                 labels={'Опыт_лет': 'Опыт работы (лет)', 'Зарплата': 'Зарплата (руб.)'})
fig.show()


In [26]:
#  Ящичковая диаграмма зависимости зарплаты от города
box_df = df.dropna(subset=['Зарплата', 'area.name'])

fig = px.box(box_df, x='area.name', y='Зарплата',
             title='Распределение зарплат по городам',
             labels={'area.name': 'Город', 'Зарплата': 'Зарплата (руб.)'})
fig.show()

In [29]:
# Гистограмма распределения зарплат
hist_df = df.dropna(subset=['Зарплата'])

# Создание гистограммы
fig = px.histogram(hist_df, x='Зарплата', nbins=50,
                   title='Распределение зарплат',
                   labels={'Зарплата': 'Зарплата (руб.)'}, color_discrete_sequence=px.colors.sequential.RdBu )
fig.show()

In [30]:
# Тепловая карта среднего зарплаты по городу и квалификации

heatmap_df = df.groupby(['area.name', 'Квалификация'])['Зарплата'].mean().reset_index()

pivot_table = heatmap_df.pivot(index='Квалификация', columns='area.name', values='Зарплата')

fig = go.Figure(data=go.Heatmap(
    z=pivot_table.values,
    x=pivot_table.columns,
    y=pivot_table.index,
    colorscale='Viridis',
    colorbar=dict(title='Средняя зарплата (руб.)')
))

fig.update_layout(title='Средняя зарплата по городу и квалификации',
                  xaxis_title='Город',
                  yaxis_title='Квалификация')
fig.show()

In [52]:
df.to_csv("vacancies.csv", index = False,  encoding='utf-8-sig')
df

,id,name,area.id,area.url,area.name,salary.from,salary.to,salary.currency,salary.gross,experience.name,salary,Зарплата
2,112312336,Инвестиционный аналитик,2,https://api.hh.ru/areas/2,Санкт-Петербург,150000.0,150000.0,RUR,False,От 1 года до 3 лет,150000.0,150000.0
3,113281092,Ассистент аналитика,1,https://api.hh.ru/areas/1,Москва,80000.0,NaN,RUR,False,Нет опыта,80000.0,80000.0
4,113740571,System Analyst/Системный Аналитик (Стажер/Junior),2,https://api.hh.ru/areas/2,Санкт-Петербург,NaN,100000.0,RUR,False,Нет опыта,100000.0,100000.0
5,109225941,Системный аналитик,1,https://api.hh.ru/areas/1,Москва,150000.0,180000.0,RUR,False,От 1 года до 3 лет,165000.0,165000.0
6,111899122,Аналитик данных,1,https://api.hh.ru/areas/1,Москва,250000.0,NaN,RUR,True,Нет опыта,217500.0,217500.0
...,...,...,...,...,...,...,...,...,...,...,...,...
94,113011701,Аналитик 1С,1,https://api.hh.ru/areas/1,Москва,250000.0,NaN,RUR,False,От 1 года до 3 лет,250000.0,250000.0
96,111732534,Аналитик по складской логистике,3,https://api.hh.ru/areas/3,Екатеринбург,60000.0,70000.0,RUR,False,Нет опыта,65000.0,65000.0
99,114039654,Химик-аналитик,1,https://api.hh.ru/areas/1,Москва,80000.0,NaN,RUR,True,Более 6 лет,69600.0,69600.0
45,112183840,Бизнес-аналитик IT (Middle),3,https://api.hh.ru/areas/3,Екатеринбург,200000.0,250000.0,RUR,False,От 1 года до 3 лет,225000.0,225000.0


In [50]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

@st.cache_data
def load_data():
    df = pd.read_csv('vacancies.csv')

    def calculate_salary(row):
        salary_from = row.get('salary.from', pd.NA)
        salary_to = row.get('salary.to', pd.NA)
        salary_gross = row.get('salary.gross', False)

        if pd.isna(salary_from) and pd.isna(salary_to):
            return pd.NA
        elif pd.isna(salary_from):
            average_salary = salary_to
        elif pd.isna(salary_to):
            average_salary = salary_from
        else:
            average_salary = (salary_from + salary_to) / 2

        coefficient = 1 if not salary_gross else 0.87
        return average_salary * coefficient

    df['Чистая_зарплата'] = df.apply(calculate_salary, axis=1)
    return df

df = load_data()

st.title('Анализ Вакансий на Должность "Аналитик"')

cities = df['area.name'].unique()
selected_cities = st.multiselect('Выберите города для отображения данных:', options=cities, default=list(cities))

filtered_df = df[df['area.name'].isin(selected_cities)].dropna(subset=['Чистая_зарплата'])

print(filtered_df)

fig = px.histogram(filtered_df, x='Чистая_зарплата', nbins=50,
                   color='area.name',
                   title='Распределение чистой зарплаты по городам',
                   labels={'Чистая_зарплата': 'Чистая зарплата (руб.)'})

st.plotly_chart(fig)


Overwriting app.py


In [53]:
from pyngrok import ngrok

ngrok.set_auth_token("2qX7M2tZ8H8BAeG4WRKnaaLpOzc_5KNFrTE5uhWvGUKGdRHYn")

!nohup streamlit run app.py --server.port 5011 &
ngrok_tunnel = ngrok.connect(addr='5011', proto='http', bind_tls=True)
print(' * Tunnel URL:', ngrok_tunnel.public_url)

nohup: appending output to 'nohup.out'
 * Tunnel URL: https://bfe7-34-73-56-187.ngrok-free.app
